In [151]:
import os
import re
import csv
import time
import pandas

In [152]:
def entries_for_each_record(input_file, output_file):
    with open (input_file) as fd:
        text = fd.read()
        section_index = {}

        for each in all_section:
            temp = text.find(each)
            if temp != -1:
                section_index[temp] = each
        print(section_index)
        sorted(section_index.items())
    boundaries = [key for key in section_index.keys()]

    fd = open(output_file,'r')
    text = fd.readlines()
    fd.close() 

    t_note_id = []
    t_section_concept_id = []
    t_offsets = []
    t_lexical_variant = []
    t_note_nlp_concept_id = []
    t_nlp_system = []
    t_nlp_date_time = []
    t_term_exists = []

    number = re.search('[0-9]{1,}',output_file)
    number = number[0]
    for each in text:
        sentence = each.split('|')
        for i in range(len(sentence)):
            if sentence[i]=='TX':
                offset_in_sentence = i+1
                concept_pos = i-1
                cui_pos = 4
        temp = sentence[offset_in_sentence].replace('[','')
        temp = temp.replace(']','')
        temp = temp.replace(',',';')
        offsets = temp.split(';')
        offsets = [aa.split('/')[0] for aa in offsets]
        length = len(offsets)
        #now we have many offsets, consider lexical_variant
        temp = sentence[concept_pos].split('-')
        concept = temp[0]
        del temp
        concept = concept.strip('[').strip('"')
        lexical_variant = [concept for a in range(length)]
        # for cui
        cui = sentence[cui_pos]
        note_nlp_concept_id = [cui for a in range(length)]
        # for note_id
        note_id = [number for a in range(length)]
        # for nlp_system
        nlp_system = ['Metamap' for a in range(length)]
        # for date
        date_time = time.strftime('%Y-%m-%d %H:%M:%S')
        nlp_date_time = [date_time for a in range(length)]
        #for term exists
        term_exists = ['Y' for a in range(length)]
        #for section concept
        section_concept_id = []
        for i in range(length):
            j = 0
            while j < len(boundaries):
                if int(offsets[i]) < boundaries[j]:
                    temp_id = j
                    break
                j+=1
                if j==len(boundaries):
                    temp_id = j
                    break
            if temp_id == 0:
                section_concept_id.append('0')
            else:
                temp_id = boundaries[temp_id-1]
                temp_section = section_index[temp_id]
                section_id = all_sections.index(temp_section)+1
                section_concept_id.append(section_id)
        # after this we have:
        # note_id, section_concept, offset, lexical, note_nlp_concept_id, nlp_system, nlp_date, term_exists
        t_note_id.extend(note_id)
        t_section_concept_id.extend(section_concept_id)
        t_offsets.extend(offsets)
        t_lexical_variant.extend(lexical_variant)
        t_note_nlp_concept_id.extend(note_nlp_concept_id)
        t_nlp_system.extend(nlp_system)
        t_nlp_date_time.extend(nlp_date_time)
        t_term_exists.extend(term_exists)

    t_snippet =  [0 for a in range(len(t_term_exists))]
    t_note_nlp_id = list(range(1,len(t_term_exists)+1))

    # write to csv file
    data_row = []
    for i in range(len(t_snippet)):
        temp = [t_note_nlp_id[i], t_note_id[i], t_section_concept_id[i], t_snippet[i], 
                    t_offsets[i], t_lexical_variant[i], t_nlp_date_time[i], t_term_exists[i]]
        data_row.append(temp)
    for each in data_row:
        writer.writerow(each)


In [153]:
columns = ['note_nlp_id','note_id','section_concept_id', 'snippet', 'offset', 
           'lexical_variant', 'note_nlp_concept_id','nlp_system','nlp_date_time', 'term_exists']
all_sections = ['Discharge Date',  'PRINCIPAL DIAGNOSIS',     'SECONDARY DIAGNOSES', 'HISTORY OF PRESENT ILLNESS',
                'PRE-ADMISSION MEDICATIONS', 'PAST MEDICAL HISTORY', 'FAMILY HISTORY',   'SOCIAL HISTORY',
                'ALLERGIES', 'ADMISSION PHYSICAL EXAMINATION',  'STUDIES:' ,'PROCEDURE',
                'HOSPITAL COURSE BY PROBLEM','COMPLICATIONS',
                'CONSULTANTS', 'PHYSICAL EXAMINATION ON DISCHARGE',
                'DISCHARGE MEDICATIONS', 'DISPOSITION', 'FOLLOW-UP APPOINTMENTS' ,'CODE STATUS', 'PRIMARY CARE PHYSICIAN']


input_dir = './input/'
output_dir = './output/'

with open('final.csv','a',newline='') as f:
    writer=csv.writer(f)
    writer.writerow(columns)
    #for eachfile in dir:
    input_files = os.listdir(input_dir)
    output_files = os.listdir(output_dir)
    for i in range(len(input_files)):
        input_file = input_dir+input_files[i]
        output_file = output_dir+output_files[i]
        entries_for_each_record(input_file,output_file)
        
    
f.close()

0
haha
{155: 'Discharge Date', 354: 'SECONDARY DIAGNOSES', 528: 'HISTORY OF PRESENT ILLNESS', 3194: 'PAST MEDICAL HISTORY', 4009: 'FAMILY HISTORY', 3879: 'SOCIAL HISTORY', 3520: 'ALLERGIES'}


In [154]:
print(len(input_files))

1
